<a href="https://colab.research.google.com/github/arputtick/dataism/blob/master/Week_4_linear_threshold_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The Linear Threshold Model**
The *linear threshold model* is widely used to simulate the diffusion of information, behavior etc. throughout a social network. It can be used in combination with the *influence maximization problem,* in which one tries to identify the most influential individuals in a social network, i.e. those from whom information diffuses the most widely.

As we observed in class, the assumptions made in the model are dubious, as are the ethics of using it, especially unbeknownst to the members of a given social network. It's also not clear that applying the LTM in such a setting as real-life social networks of homeless youth is at all effective. Though the model makes seemingly extreme simplifications about human behavior, it and similar models have at least proven effective in tasks related to efficiently diffusing information across online social networks, e.g., viral marketing. Of course it could also be used (even more) maliciously to help intentionally spread disinformation. Regardless of your own trust in the LTM, it's worth knowing about, particularly if we seek to criticize it's use.

In this notebook we review the theory and code underlying the LTM. 

---
**Note:** I've decided to slightly modify the steps as presented in class to encompass calculating the weights:

1) **Start with social network data as a directed graph:** $G = (V,E)$. Here $V$ is the set of people in the social network and $E$ is the set of pairwise connections (with direction!) between them.

2) **Compute the influence weights:** For each ordered pair of individuals $(i,j)$, we compute an influence weight $w_{ij}$.

3) **Color the 'seeds' in the graph:** These are the individuals with certain information or behavior that might be spread throughout the network.

4) **For each node, set a 'threshold value':** This is a number between 0 and 1 that determines how difficult a person is to influence. 

5) **Spread influence:** For each individual, compare the total influence on that person with their threshold. If the total influence exceeds the threshold, that individual becomes a seed.

6) **Repeat:** We repeat steps 3) and 5) as many times as there are opportunities for the nodes to influence each other. If the influence is continuous, we just repeat until no more seeds can be added.






---
# **Example**

Let's review the example from class. First we import the networkx library:

In [1]:
import networkx as nx 

# networkx as a library with functions specially suited to networks (social,
# traffic, neural etc.)

---
# 1. Start with social network data as a directed graph.

We start with a social network of six people. We represented the connections in the social network using the following *adjacency matrix:*

| Edge (from 'row' to 'column'): 0 = no edge, 1 = edge || 1|2|3|4|5|6|
|---||---|---|---|---|---|---|
|**Person 1**||0|1|1|0|1|0|
|**Person 2**||1|0|0|0|0|0|
|**Person 3**||0|1|0|0|0|0|
|**Person 4**||0|1|1|0|0|1|
|**Person 5**||0|0|1|1|0|0|
|**Person 6**||0|0|0|1|1|0|

The same social network could be given as a list of pairs:

|Person i|Person j|
|---|---|
|1|2|
|1|3|
|1|5|
|2|1|
|3|2|
|4|2|
|4|3|
|5|3|
|5|4|
|6|4|
|6|5|

Here each row corresponds to an arrow going from the person in the first column to the person in the second column.

In the real world you might come across both ways of representating social network data. The data might also include extra features that we could use in later steps, such as: 

* characteristics of each individual (e.g. whether or not they regularly use drugs), or 
* strength/type of relationship between pairs of individuals (e.g. 'strong friend' or 'aquaintance').


*In this case*, we have the extra information that:
* Person 1 and Person 3 believe Trump should be elected.
* Everyone else is undecided.
* Each of a person's influencers influences them
equally.


## Creating a graph object

First we create an empty directed graph:

In [2]:
# create directed graph object and name it 'social_net_graph'

social_net_graph = nx.DiGraph()

Then we add the edges from the above data tables:

In [3]:
social_net_graph.add_edges_from([(1,2), (1,3), (1,5), (2,1), (3,2), (4,2), (4,3), (5,3),\
                                (5,4), (6,4), (6,5)])

You can refer to the list of edges with the 'edges' function. This will be useful later, when we want to do a for-loop over the edges of the graph:

In [ ]:
social_net_graph.edges

Similarly with the nodes:

In [5]:
social_net_graph.nodes

NodeView((1, 2, 3, 5, 4, 6))

## Drawing the graph

To draw the graph, we first need to use the networkx 'spring_layout()' function to determine the position of the nodes. This uses some algorithm to find a good position for the nodes that leads to a readable graph.

In [7]:
# find the positions of nodes in the graph, as (x,y)-coordinates

pos = nx.spring_layout(social_net_graph) 

'pos' is a *dictionary* that associates to each node its corresponding position in the picture of the graph.

**Note:** Dictionaries are fundamental concepts in Python. Check out [this video](https://www.youtube.com/watch?v=uCQrfYYhQ-g) for a quick intro.

In [ ]:
pos

The most basic way to draw the graph is to use the draw_networkx( ) function:

In [ ]:
nx.draw_networkx(social_net_graph, pos)

**Note:** An arrow in the graph indicates that the person at the start of the arrow influences the person at the end of the arrow.

---
# 2) Compute the influence weights

An influence weight $w_{ij}$ should be interpreted as the amount that person $i$ influences person $j$. The sum of all the influences on a given person $j$ is a number between 0 and 1:
$$0\leq \sum_{i\mbox{ influencing $j$}} w_{ij} \leq 1 $$

In our example we assumed that the sum of these influences is *equal* to 1:

$$\sum_{i\mbox{ influencing $j$}} w_{ij} = 1 $$

We also assumed that each one of an individual's connections influences them equally:
$$w_{ij} = w_{kj},$$ where Person $i$ and Person $k$ are any pair of people influencing Person $j$.

For example, in the picture above you should be able to see that Person 3 is influenced by Person 1 and Person 4 (and no one else). This means we should have $w_{13} = w_{43} = \frac{1}{2}.$

## Compute the number of influencers each person has

The first step is to compute the number of influencers each individual in the graph has, which will give the denominators of the weights. This is done using the 'in_degree( )' function. The *in-degree* of a node is defined to be the number of arrows going *into* that node. This is exactly the number of people influencing the given person:

In [21]:
# Compute the number of influencers for each person simultaneously:
num_influencers = social_net_graph.in_degree()

## Note: I used the name 'in_deg' for num_influencers in the class demo

'num_influencers' is now a dictionary that associates to each person the number of people that are influencing it. You can double-check in the picture above to make sure.

In [ ]:
# Check what num_influencers looks like
num_influencers

## Compute the weights

Let $n_j$ be the number of people influencing person $j$. Since each of the people influencing person $j$ do so with equal weight, they should each have an influence weight of $\frac{1}{n_j}$. In other words, we should have 
$$ w_{ij} = \frac{1}{n_j},$$
for every person $i$ who influences person $j$. Compare this general formula to the example with $j = 3$ and $ i = 1,4$ above.

The following code computes the weights. Remember that every edge in the graph gets a corresponding weight. Our code labels each edge with it's weight.

For example, to label the edge (1,3) with the weight $w_{13} = \frac{1}{2}$, you would write


```
social_net_graph.edges[1,3]['influence'] = 1 / 2
```

When we write that, we are simultaneously creating an attribute for all of the edges called 'influence' and setting the influence for the (1,3) edge equal to 1/2.

In [23]:
## Compute the influence weights and label the edges with them under the
## attribute name 'influence.'

for edge in social_net_graph.edges:
    social_net_graph.edges[edge[0],edge[1]]['influence'] = 1 / num_influencers[edge[1]]
    # edge[0] is where the edge starts and edge[1] is where the edge ends.

We can print out the computed weights to make sure it worked as expected:

In [ ]:
for edge in social_net_graph.edges:
    print(edge, social_net_graph[edge[0]][edge[1]]['influence'])
    # Note: 'social_net_graph[edge[0]][edge[1]]' and 'social_net_graph.edges[edge[0],edge[1]]'
    # are equivalent. Use whichever you find more intuitive.

# 3) Color the seeds in the graph

In this example, the seeds are the people who believe Trump should be re-elected. They are trying to spread this view to others in their social network.

We start with Person 1 and Person 3 as seeds. It'll be useful to keep track of the non-seeds, i.e., everyone else as well. Why? Because we might want to represent them with a different color or consider the more complicated situation in which two types of influence compete against each other (such as users vs. non-users in the USC study or vote Trump vs. no Trump in an extension of this example).

In [25]:
seeds = [1,3]
non_seeds = [2,4,5,6]

Let's visualize the seeds and non-seeds in our graph:

In [ ]:
# First we draw the graph with only the seeds colored (red).
# The nodes we wish to color are given by 'nodelist.'
nx.draw_networkx(social_net_graph, pos, 
                    nodelist = seeds,
                    node_color = 'r') # 'r' stands for red

# We can also color the non_seeds if we run the following
# in the SAME CODE CELL.
nx.draw_networkx(social_net_graph, pos, 
                    nodelist = non_seeds,
                    node_color = 'w') # 'w' stands for white

---
# 4) Set the threshold values

Each person $i$ in the social network gets a corresponding threshold $\tau_i$ between 0 and 1 which encodes how difficult a person is to influence. $\tau_i = 0$ means person $i$ is a sheep and $\tau_i = 1$ means person $i$ only changes their mind if all of their connections adopt a new opinion.

**We assume for simplicity that every individual has a threshold of 0.5.**

**Note:** Giving everyone the same threshold value is sometimes done in practice, but in the USC study, the thresholds are chosen randomly *from a uniform distribution*. This just means that every number between 0 and 1 is equally likely (as opposed to, say, a normal ditribution, where numbers close to a certain average value are more likely).

**Note:** I denoted $\tau_i$ by $\mbox{Threshold}(i)$ in class.

**Note:** If person $i$ will *never* change their mind, no matter what happenes in their social network, you can set $\tau_i > 1$, e.g. $\tau_i = 1.1$.

In [26]:
# Give every person a threshold value of 0.5
for node in social_net_graph.nodes:
    social_net_graph.nodes[node]['threshold'] = 0.5

**Note:** Just as we created a new attribute for the edges called 'influence' and stored the influence weights there, we just created an attribute for the nodes called 'threshold' and stored the threshold values there. We could use any names we want in place of 'influence' or 'threshold,' but it's good practice to pick as clear a name as possible.

In [ ]:
# Check Person 1's threshold value:
social_net_graph.nodes[1]['threshold']

---
# 5) Spread influence

For each person $i$ that *isn't* a seed, take the sum
$$\mbox{influence}(i) = \sum_{\mbox{neighboring seeds}} w_{ij}.$$

If $\mbox{influence}(i) > \tau_i,$ then vertex $i$ becomes a seed, i.e., adopts the behavior/info that is being propagated through the social network from the original set of seeds.

In our example, this means that we look at each of the pro-Trump people influencing a given person. If the sum of their influence weights is greater that the threshold (0.5), then that person becomes pro-Trump as well.

In [28]:
## influence spread

# First create a an empty list to keep track of who becomes pro-Trump this round.
# We do this so that we don't end up counting new seeds in the following for loop.
# This is because the assumption is that they haven't had a chance to spread their
# opinion yet.
new_seeds = []

# Now we compare the total influence with the threshold for every person 
# in the social network.

# The most logical way to do this is with a for-loop that runs over the nodes:
for node in social_net_graph.nodes:
    # We only do something of the node is NOT a seed.
    if node in seeds:
        continue

    else:
        # First create a variable to store the total influence
        # This is the influence(i) number in the above text.
        influence_sum = 0

        # Now we look at all the influencers of the node we're currently looking at.
        # This is done using the 'predecessors()' function, which takes a node-index
        # as input and outputs a list of all of the nodes influencing it.
        # We do another for-loop over this list of influencers.
        for neighbor in social_net_graph.predecessors(node):

            # We only add the influence weights to influence_sum that correspond
            # to seeds:
            if neighbor in seeds:
                influence_sum += social_net_graph.edges[neighbor,node]['influence']

        # At the end of this inner for-loop, influence_sum is equal to the desired
        # sum.

        
        # We now compare the total influence to the person's threshold value
        if influence_sum > social_net_graph.nodes[node]['threshold']:
            # If the influence exceeds the threshold, we add the node to our new_seeds
            # list.
            new_seeds.append(node)

            # We also remove the node from the list of non_seeds
            non_seeds.remove(node)


# Now that we've performed one round of influence spreading, we add all of the new seeds
# to our list of seeds
seeds += new_seeds

Let's check which people now believe that Trump should be re-elected:

In [ ]:
seeds

We can also visualize the results of the influence spread:

In [ ]:
nx.draw_networkx(social_net_graph, pos, 
                    nodelist = seeds,
                    node_color = 'r')

nx.draw_networkx(social_net_graph, pos, 
                    nodelist = non_seeds,
                    node_color = 'w')

# 6) Repeat

**Exercise:** Try rerunning the influence spread loop above. You should see that no new seeds have been added.

**Exercise:** Now try going back and changing the initial set of seeds as follows:


```
seeds = [1,3,6]
non_seeds = [2,4,5]
```
You should see that everyone wants to vote for Trump after two rounds of influence spread.
